In [1]:
import robotic as ry
import numpy as np
import time
import random

ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_add({'botsim/engine': 'physx'}) #makes a big difference!
ry.params_add({'physx/multibody': True}) #makes a big difference!

C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.addFile("maze_obs_move_env.g")

C.computeCollisions()
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

C3 = ry.Config()
C3.copy(C)

bot = ry.BotOp(C, useRealRobot=False)

def IK(C, target, qHome):
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 0, False) #one phase one time slice problem, with 'delta_t=1', order=0
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) #cost: close to 'current state'
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
    komo.addObjective([], ry.FS.poseDiff, ['l_gripper', target], ry.OT.eq, [1e2]) #constraint: gripper position
    ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    return [komo.getPath()[0], ret]

def IK_path(C, target, qHome):
    skeleton = ry.Skeleton()
    skeleton.addEntry([1, -1], ry.SY.contactStick,  ["l_gripper", "pawn_handle"])
    skeleton.addEntry([1, -1], ry.SY.stable,  ["l_gripper", "pawn_handle"])
    skeleton.addEntry([1, 2], ry.SY.poseEq, ["l_gripper", target])
    skeleton.enableAccumulatedCollisions(True)

    komo = skeleton.getKomo_waypoints(C, lenScale=1e-1, homingScale=1e-2, collScale=1e1)
    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=1, stopEvals=10000, maxStep=100).solve()

    waypoints = komo.getPath_qAll()

    komo.view(True, "waypoints solution")
    komo.view_play(True, 0.1)
    return [komo.getPath()[0], ret]

def RRT(qF, target, rrt_extend_length=0.04):
    C2 = ry.Config()
    C2.addFile(target + "_actuated.g")
    q0 = C2.getJointState()
    print("Start State: ", q0," Final State: ", qF)
    C2.view()

    # Create an RRT path finder
    rrt = ry.PathFinder()
    rrt.setProblem(C2, [q0], [qF])
    
    isFeas = False
    path = None

    i=0
    # Find an RRT path
    while i!=10 and (not isFeas):
        rrt = ry.PathFinder()
        rrt.setProblem(C2, [q0], [qF])
        ret = rrt.solve()
        isFeas = ret.feasible
        path = ret.x
        i+=1
    

    ## Smooth the path (optimization) ##
    print("Path Prev:", len(path))
    #Create a KOMO for post-optimization with acceleration constraints
    num_time_steps = len(path)

    komo2 = ry.KOMO(C2, num_time_steps, 1, 2, True)
    #Add an acceleration constraint objective to the new KOMO object
    komo2.addControlObjective([], 2, 1e0)
    komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e2])
    #Set the waypoints for the KOMO
    komo2.initWithPath_qOrg(path)

    ret = ry.NLP_Solver() \
        .setProblem(komo2.nlp()) \
        .setOptions(stopTolerance=1e-2, verbose=0) \
        .solve()
    print(ret)

    path = komo2.getPath()
    print("Path Post:", len(path))

    new_path = path

    # for i in range(0, len(new_path)):
    #     C2.setJointState(new_path[i])
    #     C2.view()
    #     time.sleep(.5)
    
    return new_path

def calculateContactPoint(middle_point, target_point, size):
    """
    This function is used to find the contact point to move the target object to the designated goal. 
    """
    radius = size[0] * 2
    # Calculate the vector from the middle point to the near point
    vector = target_point - middle_point
    # Normalize the vector to make it a unit vector
    normalized_vector = vector / np.linalg.norm(vector)
    # Calculate the point on the opposite side of the circle
    opposite_point = middle_point - normalized_vector * radius
    return opposite_point 

def addMarker(C, pos, name, parent, size=0.1):
    marker = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.ssBox, size = [0, 0, 0, .001]) \
              .setPosition(pos)
    C.view()
    return marker

def addPoint(C, pos, name, parent, color):
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setPosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

def addPoint2(C, obj, name, parent, color):
    pos = C.frame(obj).getPosition()
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setPosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

def addBox(C, name, pos,parent):
        size = [0.04, .04, .04, .0]
        C.addFrame(name = name,parent=parent) \
            .setShape(ry.ST.ssBox, size = size) \
            .setRelativePosition(pos) \
            .setColor([0, 1, 1]) \
            .setContact(True)\
            .setMass(0.1)

def moveBotAlongPath(C, bot, path, contact_point="", obstacle_name=""):
    qHome = bot.get_qHome()
    
    addPoint(C, C.frame("cpoint").getPosition(), "contact"+str(random.randint(0,10000)), "pawn_handle", [1, 1, 0])
    q0, ret = IK(C, "cpoint", qHome)
    bot.moveTo(q0, timeCost=3., overwrite=True)
    bot.sync(C, 1) #keep the workspace C sync'ed to real/sim, and idle .1 sec

    bot.gripperMove(ry._left, width=.005, speed=.2)
    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

    if contact_point != "":

        q1, ret = IK(C, "cpoint2", qHome)
        bot.moveTo(q1, timeCost=3., overwrite=True)
        bot.sync(C, .1) #keep the workspace C sync'ed to real/sim, and idle .1 sec

        cp = C.frame(contact_point)
        cp.setPosition(cp.getPosition() + np.array([0,0,.3]))

        
        q_target, ret = IK(C, contact_point, qHome)
        print(q_target)
        bot.moveTo(q_target, timeCost=5, overwrite=True)
        while bot.getTimeToEnd()>0:
            bot.sync(C, .1)

        cp.setPosition(cp.getPosition() + np.array([0,0,-.3]))

        q_target, ret = IK(C, contact_point, qHome)
        print(q_target)
        bot.moveTo(q_target, timeCost=5, overwrite=True)
        while bot.getTimeToEnd()>0:
            bot.sync(C, .1)
   
        for i in path:
            addMarker(C, [*i[:2], 0.79], "target", obstacle_name, 0.07)
            q_target2, ret = IK(C, "target", qHome)
            bot.moveTo(q_target2, timeCost=.3, overwrite=True)
            while bot.getTimeToEnd()>0:
                bot.sync(C, .1)
            addPoint(C, C.frame(obstacle_name).getPosition(), "contact"+str(random.randint(0,10000)), "world", [1, 0, 0])

        addMarker(C, [*path[-1][:2], 1], "target", obstacle_name, 0.07)
        q_target2, ret = IK(C, "target", qHome)
        bot.moveTo(q_target2, timeCost=.3, overwrite=True)
        while bot.getTimeToEnd()>0:
            bot.sync(C, .1)


        bot.moveTo(q1)
        bot.sync(C, .1)

        bot.moveTo(q0)
        bot.sync(C, .1)
            
    else:
         for i in path:
            addMarker(C, [*i[:2], 0.79], "target", "box", 0.07)
            q_target2, ret = IK(C, "target", qHome)
            bot.moveTo(q_target2, timeCost=.3, overwrite=True)
            while bot.getTimeToEnd()>0:
                bot.sync(C, .1)

            addPoint(C, C.frame("box").getPosition(), "contact"+str(random.randint(0,10000)), "world", [1, 0, 0])

def moveBotAlongPath2(C, bot, path, contact_point="", obstacle_name=""):
    qHome = bot.get_qHome()

    cp = C.frame(contact_point)
    cp.setPosition(cp.getPosition() + np.array([0,0,.3]))
    q_target, ret = IK(C, contact_point, qHome)
    bot.moveTo(q_target, timeCost=5., overwrite=True)
    while bot.getTimeToEnd()>0:
        bot.sync(C, .1)

    cp.setPosition(cp.getPosition() + np.array([0,0,-.3]))
    q_target, ret = IK(C, contact_point, qHome)
    bot.moveTo(q_target, timeCost=5., overwrite=True)
    while bot.getTimeToEnd()>0:
        bot.sync(C, .1) 

    for i in path:
        addMarker(C, [*i[:2], 0.79], "target", obstacle_name, 0.07)
        q_target2, ret = IK(C, "target", qHome)
        bot.moveTo(q_target2, timeCost=.1, overwrite=True)
        while bot.getTimeToEnd()>0:
            bot.sync(C, .1)
        addPoint(C, C.frame(obstacle_name).getPosition(), "contact"+str(random.randint(0,10000)), "world", [1, 0, 0])

def moveToFreeSpace(C, C3, bot, path):
    """
    Moves the obstacle to free space
    """
    target_pos = dict()
    path_obs   = dict()
    obstacle_name = colCheck(C3, path)
    while(obstacle_name != None):
        print("There is collision with ", obstacle_name)
        if(obstacle_name == "obstacle_1"):
            new_pos = [-0.4, 0.28, 0] #TODO: Calculate it with model
        elif (obstacle_name == "obstacle_2"):
            new_pos = [-0.3, 0.28, 0] #TODO: Calculate it with model
        elif (obstacle_name == "obstacle_3"):
            new_pos = [-0.2, 0.28, 0] #TODO: Calculate it with model
        else:
            new_pos = [-0.4, 0.28, 0] #TODO: Calculate it with model
        target_pos[obstacle_name] = new_pos
        o = C3.frame(obstacle_name)
        o.setPosition([*new_pos[:2], 0.68])
        obstacle_name = colCheck(C3, path)
        
    for obstacle_name, new_pos in target_pos.items():
        print("Removing ", obstacle_name, " from the path...")
        obstacle = C.frame(obstacle_name)
        path_obs = RRT(new_pos, obstacle_name, rrt_extend_length=0.01)
        print(path_obs[-1])
        target_point = [*path_obs[-1][:2], 0.68001]
        addMarker(C, target_point, "goal_" + obstacle_name, "box", 0.05)

        if(obstacle_name == "obstacle_1"):
            contact_point_obstacle=[-0.4,  0.5, 0.79] #TODO: Calculate it with model
        elif (obstacle_name == "obstacle_2"):
            contact_point_obstacle=[-0.3,  0.5, 0.79] #TODO: Calculate it with model
        elif (obstacle_name == "obstacle_3"):
            contact_point_obstacle=[-0.2,  0.5, 0.79] #TODO: Calculate it with model
        else:
            contact_point_obstacle=[-0.4,  0.5, 0.79] #TODO: Calculate it with model
            
        addMarker(C, contact_point_obstacle, "contact_point_obstacle", "world", 0.1)
        print("Middle point:", obstacle.getPosition(), "Target Point: ", target_point, "Contact Point Obstacle", contact_point_obstacle)
        #C.view(True)
        moveBotAlongPath(C, bot, path_obs, "contact_point_obstacle", obstacle_name)
        while(bot.getTimeToEnd() > 0): 
            bot.sync(C, 0.1)
        
    contact_point_box=[0.1,  0.4, 0.79] #TODO: Calculate it with model
    addMarker(C, contact_point_box, "contact_point_box", "world", 0.1)
    moveBotAlongPath2(C, bot, path, "contact_point_box", "box")

def colCheck(C, path):
    for i in range(1, len(path)):
        addBox(C,"shell_"+str(i),[*path[i][:2],0.68],"world")
        
        C.computeCollisions()
        collision = C.getCollisions()

        for j in range(0, len(collision)):
            if(collision[j][0].split("_")[0] == "shell" and collision[j][1].split("_")[0] == "obstacle"):
                C.delFrame("shell_"+str(i))
                return collision[j][1]

        C.delFrame("shell_"+str(i))
    return None
    

-- WARNING:kin_physx.cpp:prepareLinkShapes:627(-1) computing compound inertia for object frame 'world' -- this should have been done earlier?


In [2]:
qF = [-0.54,0.4,0.68]
addMarker(C,qF,'marker','world')
path = RRT([*qF[:2], 0], "box")

Start State:  [0.  0.4 0. ]  Final State:  [-0.54, 0.4, 0]
Path Prev: 13
{ time: 0.000778, evals: 4, done: 1, feasible: 1, sos: 0.000631892, f: 0, ineq: 0, eq: 0 }
Path Post: 13


In [3]:
moveToFreeSpace(C, C3, bot, path)

There is collision with  obstacle_3
There is collision with  obstacle_3
There is collision with  obstacle_2
There is collision with  obstacle_2
There is collision with  obstacle_1
There is collision with  obstacle_1
Removing  obstacle_3  from the path...
Start State:  [-0.2  0.4  0. ]  Final State:  [-0.2, 0.28, 0]
Path Prev: 3
{ time: 0.001937, evals: 5, done: 1, feasible: 1, sos: 0.000447551, f: 0, ineq: 0, eq: 0 }
Path Post: 3
[-0.2032219   0.32180543 -0.00939091]
Middle point: [-0.2   0.4   0.68] Target Point:  [-0.2032219046535302, 0.32180542546203783, 0.68001] Contact Point Obstacle [-0.2, 0.5, 0.79]
[ 0.25785011  0.79718491  0.0571074  -0.5517492  -0.04187636  1.3483112
  1.07392692]
[ 0.24483391  0.75202069  0.04395912 -1.41597673 -0.0362918   2.16721594
  1.08273871]
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:ki

In [4]:
for i in C.getFrameNames():
    if "panda" in i or i in ["gripper", "palm", "finger1", "finger2", "l_gripper", "l_palm", "l_finger1", "l_finger2", "cameraTop", "cameraWrist", "bellybutton"]:
        C.delFrame(i)
C.view()

0

In [5]:
C.getFrameNames()

['world',
 'table',
 'world',
 'wall',
 'bin_side',
 'bin_side2',
 'bin_side3',
 'bin_side4',
 'box',
 'wpoint',
 'pawn',
 'pawn_handle',
 'cpoint',
 'cpoint2',
 'obstacle_1',
 'obstacle_2',
 'obstacle_3',
 'marker',
 'goal_obstacle_3',
 'contact_point_obstacle',
 'contact805',
 'target',
 'contact7007',
 'contact835',
 'contact2393',
 'goal_obstacle_2',
 'contact8882',
 'contact6877',
 'contact387',
 'contact9522',
 'contact395',
 'goal_obstacle_1',
 'contact5644',
 'contact8858',
 'contact5537',
 'contact2531',
 'contact3215',
 'contact5478',
 'contact_point_box',
 'contact6002',
 'contact2016',
 'contact5298',
 'contact545',
 'contact3810',
 'contact8695',
 'contact1160',
 'contact1077',
 'contact746',
 'contact3536',
 'contact941',
 'contact3132',
 'contact5369']

In [6]:
import robotic as ry
import numpy as np
import time
import random

ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_add({'botsim/engine': 'physx'}) #makes a big difference!
ry.params_add({'physx/multibody': True}) #makes a big difference!

C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.addFile("maze_obs_move_env.g")
C.delFrame("cameraWrist")
C.computeCollisions()
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0